# **Lexicon**

In [1]:
import numpy as np
import pandas as pd

### Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### DB 연결
# !pip install pymysql
import pymysql

### DB에 저장
import sqlalchemy
from sqlalchemy import create_engine

### 실시간 주식가격 데이터
# !pip install finance-datareader
import FinanceDataReader as fdr

### 텍스트 분석
## KoNLPy
# 1) JAVA 설치, 2) Python 버전과 맞는 JPype1-py3 설치, 3) !pip install konlpy, 4) 설치 경로에서 jvm.py 파일 코드 67번 줄 주석 처리 
from konlpy.tag import Okt
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
## FastText
# !pip install gensim
# !pip install fasttext
import fasttext
import fasttext.util
# Facebook 한국어 Embedding 모델 다운로드 → 한 번만 설치하면 됨
fasttext.util.download_model('ko', if_exists='ignore')
# 유사도 계산
from gensim import models

### 모델 학습 및 평가
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import precision_score, recall_score, f1_score

### 모델 저장 및 로드
import joblib

### 기타
import datetime
from collections import Counter
import sys
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf

print('GPU', '사용 가능' if tf.config.experimental.list_physical_devices('GPU') else '사용 불가능')

GPU 사용 불가능


In [3]:
!nvidia-smi

Sun Sep 19 02:01:10 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 457.49       Driver Version: 457.49       CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 1650   WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   46C    P8     1W /  N/A |    460MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## **Read KOSELF & Test Data**

### **① KOSELF 감성 어휘 사전**

In [4]:
# 블로그에서 가져온 기본적인 한국어 긍부정 텍스트 목록
with open('positive_words_self.txt', encoding='utf-8') as pos_blog:
    positive_blog = pos_blog.readlines()
positive_blog = [pos_blog.replace('\n', '') for pos_blog in positive_blog]
with open('negative_words_self.txt', encoding='utf-8') as neg_blog:
    negative_blog = neg_blog.readlines()
negative_blog = [neg_blog.replace('\n', '') for neg_blog in negative_blog]

# KOSELF 감성 어휘 사전
with open('KOSELF_pos.txt', encoding='utf-8') as pos:
    positive = pos.readlines()
positive = [pos.replace('\n', '') for pos in positive]
with open('KOSELF_neg.txt', encoding='utf-8') as neg:
    negative = neg.readlines()
negative = [neg.replace('\n', '') for neg in negative]

### **② 2018년 SK Data**

In [5]:
lexicon_2018 = pd.read_csv('../../../../Code/Data/Test/Stock-Year/2018_sk.csv')

lexicon_2018.head(1)

,st_n,st_cd,news,datetime,title,url,text,date,time,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed,Tokenization
0,sk하이닉스,660,매일경제,2018010100,"동원산업·휠라코리아 ""생큐 트럼프""",http://news.mk.co.kr/newsRead.php?year=2018&no=30,■ 2018 美법인세인하 수혜株 \n\n\n 올해 1월부터 법인세 인하를 골자로 한...,2018-01-02,0,77300,77300,76200,76600,2014838,0.001307,1,0,법인세 인하 수혜 올해 법인세 인하 골자 미국 세제 개혁 법안 현실 국내 증시 관련...


In [6]:
lexicon_2018[lexicon_2018['text'].isna()]

,st_n,st_cd,news,datetime,title,url,text,date,time,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed,Tokenization


In [7]:
lexicon_2018.dropna(axis=0, inplace=True)
lexicon_2018 = lexicon_2018.reset_index(drop=True)
lexicon_2018.isna().sum()

st_n                 0
st_cd                0
news                 0
datetime             0
title                0
url                  0
text                 0
date                 0
time                 0
Open                 0
High                 0
Low                  0
Close                0
Volume               0
Change               0
UpDown               0
Extremely_Changed    0
Tokenization         0
dtype: int64

## **Calculate Cosine Similarity**

In [8]:
# 이미 학습된 FastText 내장 한국어 모델
try:
    print(ko_model, '모델 로드 불필요')
except:
    ko_model = models.fasttext.load_facebook_model('cc.ko.300.bin')

In [9]:
limit_number = 15000
sys.setrecursionlimit(limit_number)

In [10]:
pos_lexicon_2018 = pd.DataFrame(columns=['date', 'news_num', 'KOSELF_pos_word', 'news_word', 'cosine_similarity', 'frequency'])   # 'frequency' 구현할 방법은 추후 생각...ㅠㅠ

# KOSELF_pos와의 Cosine Similarity 계산
for x in range(len(lexicon_2018['Tokenization'])):
    news_num = x+1
    for y in range(len(positive)):
        for z in range(len(list(set(lexicon_2018['Tokenization'][x].split())))):
            if (ko_model.wv.similarity(positive[y], list(set(lexicon_2018['Tokenization'][x].split()))[z]) >= 0.5) and (ko_model.wv.similarity(positive[y], list(set(lexicon_2018['Tokenization'][x].split()))[z]) != 1.0):
                freq = 0
                for w in range(len(lexicon_2018['Tokenization'][x].split())):
                    if lexicon_2018['Tokenization'][x].split()[w] == list(set(lexicon_2018['Tokenization'][x].split()))[z]:
                        freq += 1
                data = {
                    'date': lexicon_2018['date'][x],
                    'news_num': news_num,
                    'KOSELF_pos_word': positive[y],
                    'news_word': list(set(lexicon_2018['Tokenization'][x].split()))[z],
                    'cosine_similarity': ko_model.wv.similarity(positive[y], list(set(lexicon_2018['Tokenization'][x].split()))[z]),
                    'frequency': freq
                    }
                pos_lexicon_2018 = pos_lexicon_2018.append(data, ignore_index=True)
                print("***{0} Cosine Similarity between <{1}> & <{2}> : {3}".format(lexicon_2018['date'][x], positive[y], list(set(lexicon_2018['Tokenization'][x].split()))[z], ko_model.wv.similarity(positive[y], list(set(lexicon_2018['Tokenization'][x].split()))[z])))

***2018-01-02 Cosine Similarity between <경신> & <기록> : 0.522990882396698
***2018-01-02 Cosine Similarity between <의심의 여지가 없는> & <자공시> : 0.518829345703125
***2018-01-02 Cosine Similarity between <증대> & <증가> : 0.6946426630020142
***2018-01-02 Cosine Similarity between <추진> & <중인> : 0.539746880531311
***2018-01-02 Cosine Similarity between <호황> & <훈풍> : 0.5522077083587646
***2018-01-02 Cosine Similarity between <본격적인> & <본격> : 0.6104379296302795
***2018-01-02 Cosine Similarity between <증대> & <급증> : 0.5051040649414062
***2018-01-02 Cosine Similarity between <확보> & <마련> : 0.5137236714363098
***2018-01-02 Cosine Similarity between <강세> & <약세> : 0.7028064727783203
***2018-01-02 Cosine Similarity between <개선> & <방안> : 0.5635414123535156
***2018-01-02 Cosine Similarity between <개선> & <개선> : 0.9999999403953552
***2018-01-02 Cosine Similarity between <실현> & <차익> : 0.531851053237915
***2018-01-02 Cosine Similarity between <우세> & <강세> : 0.5120405554771423
***2018-01-02 Cosine Similarity between <중요한

In [11]:
pos_lexicon_2018

,date,news_num,KOSELF_pos_word,news_word,cosine_similarity,frequency
0,2018-01-02,1,경신,기록,0.522991,1
1,2018-01-02,1,의심의 여지가 없는,자공시,0.518829,1
2,2018-01-02,1,증대,증가,0.694643,3
3,2018-01-02,1,추진,중인,0.539747,1
4,2018-01-02,1,호황,훈풍,0.552208,1
...,...,...,...,...,...,...
12650,2019-01-02,2603,중요한,가장,0.537125,1
12651,2019-01-02,2603,증대,증가,0.694643,1
12652,2019-01-02,2603,추진,계획,0.520543,2
12653,2019-01-02,2603,호황,업황,0.554454,1


In [12]:
pos_lexicon_2018.to_csv('../../../../Code/Data/Test/Stock-Year/pos_sk_2018.csv', index=False)

In [13]:
a = list(set(list(pos_lexicon_2018['news_word'])))

a

['감소',
 '신윤근',
 '확대',
 '요구',
 '겹악재',
 '본격',
 '촉진',
 '확충',
 '드래콘',
 '뎬나오뎬',
 '넷플릿스',
 '팽팽하',
 '견조할',
 '극복',
 '최선',
 '과잉반응',
 '활기',
 '기록',
 '퍼부으',
 '증가',
 '롱텀펀드',
 '강점',
 '장단점',
 '종금업',
 '장점',
 '시너지',
 '송년간담회',
 '웰바이오텍',
 '캐펙스',
 '단점',
 '코팅덕트',
 '해법',
 '최진명',
 '문제',
 '훈풍',
 '사업',
 '검토',
 '방안',
 '코스포켐텍',
 '불경기',
 '이유섭',
 '프론텍',
 '불황',
 '상쇄하',
 '일진일퇴',
 '이점',
 '덧붙엿다',
 '벽체',
 '실패',
 '보락보락',
 '광풍',
 '공영은',
 '고취',
 '엄주용',
 '세릍리온',
 '팔리흐',
 '숭요처',
 '증대',
 '박유악',
 '선박물량',
 '둥팡밍주탑',
 '특성',
 '매렸다',
 '노스캘리포니아',
 '활황',
 '보통신',
 '웃돌',
 '횡보세',
 '강력',
 '충족',
 '젬백스젬백스',
 '약세',
 '강세',
 '롱액팅',
 '배규웅',
 '급락',
 '계획',
 '중거리핵전력조약',
 '우호',
 '프럼파스트',
 '맥쿼리인프라',
 '강도희',
 '심병창',
 '저하',
 '엠에스',
 '김민현',
 '구현',
 '다운턴',
 '최고급',
 '마그비',
 '소재효',
 '첵랍콕국제공항',
 '맞소송',
 '확립',
 '언택트',
 '차익',
 '의미',
 '웃돈',
 '니케이지수',
 '롱숏펀드',
 '가장',
 '횡보할',
 '기가와트',
 '횡보등락할',
 '최봉길',
 '캔서롭캔서롭',
 '슈퍼샘플링',
 '닛케이지수',
 '토픽스지수',
 '문유진',
 '마련',
 '확보',
 '것이므',
 '진행',
 '밑돌',
 '루브리컨츠',
 '조호윤',
 '티플랙스',
 '집중근무제',
 '업황',
 '페드워치',
 '루텍재영솔루텍',
 '폭락',
 '

In [14]:
b = list(set(list(pos_lexicon_2018[pos_lexicon_2018['cosine_similarity']>=0.7]['news_word'])))

b

['밑돈',
 '쎄미켐동진쎄미켐',
 '실패',
 '경의',
 '추진',
 '장점',
 '개선',
 '충족',
 '단점',
 '최선',
 '약세',
 '불황',
 '우월']

In [15]:
c = list(set(list(pos_lexicon_2018[pos_lexicon_2018['cosine_similarity']>=0.65]['news_word'])))

c

['감소',
 '밑돈',
 '김민현',
 '개선',
 '맞소송',
 '불황',
 '실패',
 '적극',
 '숭요처',
 '최선',
 '경의',
 '증가',
 '웃돌',
 '장점',
 '충족',
 '것이므',
 '약세',
 '쎄미켐동진쎄미켐',
 '추진',
 '단점',
 '리비젼',
 '우월']